In [23]:
import torch
import math
import transformers
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, GPT2Tokenizer, AutoConfig, AutoModelForCausalLM

In [24]:
dataset = load_from_disk("./sample_dataset")
dataset = dataset.train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 309
    })
    test: Dataset({
        features: ['text'],
        num_rows: 78
    })
})

In [26]:
context_length = 64
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
outputs = tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)
print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs)}")
# print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 2
Input chunk lengths: {'overflowing_tokens': [[287, 281, 15452, 1182, 198, 20451, 278, 514, 1363, 198, 198, 40, 2911, 314, 1239, 4425, 345, 198, 34456, 340, 1239, 5645, 198, 40, 1549, 1239, 2513, 11424, 25418, 3530, 757, 198, 2504, 338, 262, 17855, 2612, 9032, 640, 714, 1239, 47618, 198, 40, 1549, 1239, 2513, 11424, 25418, 3530, 757, 198, 1870, 5156, 11, 314, 651, 21619, 1431, 416, 703, 198, 1212, 1748, 26557, 534, 1438, 198, 1870, 5156, 11, 314, 1101, 523, 22144, 286, 198, 361, 345, 1683, 2513, 1497, 198, 40, 1549, 1239, 2513, 11424, 25418, 3530, 757, 198, 40, 1549, 1239, 2513, 11424, 25418, 3530, 757, 198, 198, 11209, 28507, 826, 1280, 11, 23608, 1633, 198, 41, 8317, 705, 744, 616, 12450, 318, 12431, 198, 1135, 12012, 262, 29424, 319, 11424, 25418, 3530, 198, 13579, 273, 1096, 262, 1126, 4730, 287, 262, 4314, 198, 7282, 618, 356, 547, 2657, 27476, 11, 2712, 1830, 198, 40, 1807, 345, 547, 3756, 502, 319, 198, 40, 11856, 616, 11668, 11, 1364, 11424, 25418, 3530, 198,

In [27]:
# def tokenize(element):
#     outputs = tokenizer(
#         element["text"],
#         truncation=True,
#         max_length=context_length,
#         return_overflowing_tokens=True,
#         return_length=True,
#     )
#     input_batch = []
#     for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
#         if length == context_length:
#             input_batch.append(input_ids)
#     return {"input_ids": input_batch}

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=64)

tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=dataset["train"].column_names
)

# block_size = int(tokenizer.model_max_length / 4)
block_size = 64
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

print(tokenizer.decode(lm_datasets['train'][65]['input_ids']))
print(tokenizer.decode(lm_datasets['train'][65]['labels']))

Someone struck a match against the night
All I could see was you and I
It was captivating
A perfect little dream inside my head
And then reality crept in
And erased it
For a while I thought that I could hold you
But you were just a temporary high

Firefly
You
Someone struck a match against the night
All I could see was you and I
It was captivating
A perfect little dream inside my head
And then reality crept in
And erased it
For a while I thought that I could hold you
But you were just a temporary high

Firefly
You


In [29]:
config = AutoConfig.from_pretrained('distilgpt2', vocab_size=len(tokenizer), n_ctx=context_length, bos_token_id=tokenizer.bos_token_id, eos_token_id=tokenizer.eos_token_id)

model = AutoModelForCausalLM.from_pretrained("output/model-v4/checkpoint-4340/")

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [30]:
out = data_collator([lm_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")


input_ids shape: torch.Size([5, 64])
attention_mask shape: torch.Size([5, 64])
labels shape: torch.Size([5, 64])


In [31]:
from transformers import Trainer, TrainingArguments
import wandb
import random
seed_data = random.randint(0,2**32-1)
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1
args = TrainingArguments(
    output_dir="output/model-generator",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=1,
    seed=seed_data,
    do_eval=True,
    eval_steps=1,
    logging_steps=5,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=1.372e-4,
    push_to_hub=True,
    report_to='wandb',
    resume_from_checkpoint=True,
    load_best_model_at_end=True,
    use_mps_device=True
)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)


from transformers import get_cosine_schedule_with_warmup
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
      trainer.optimizer,
      num_warmup_steps=0,
      num_training_steps=num_train_steps
)

env: PYTORCH_ENABLE_MPS_FALLBACK=1


/Users/bhavya/projects/hugging-face-playground/output/model-generator is already a clone of https://huggingface.co/BhavyaMuni/model-generator. Make sure you pull the latest changes with `repo.git_pull()`.
/opt/homebrew/Caskroom/miniforge/base/envs/lyrics/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
trainer.train()

  1%|▏         | 5/390 [00:02<02:38,  2.43it/s]

{'loss': 3.3589, 'learning_rate': 0.00013171058983499535, 'epoch': 0.13}


  3%|▎         | 10/390 [00:04<01:57,  3.23it/s]

{'loss': 2.5818, 'learning_rate': 0.00011612089065075853, 'epoch': 0.26}


  4%|▍         | 15/390 [00:05<01:50,  3.39it/s]

...


100%|██████████| 390/390 [02:35<00:00,  3.32it/s]

{'eval_loss': 2.3939199447631836, 'eval_runtime': 0.5437, 'eval_samples_per_second': 143.464, 'eval_steps_per_second': 18.393, 'epoch': 10.0}


100%|██████████| 390/390 [02:37<00:00,  2.48it/s]

{'train_runtime': 157.4999, 'train_samples_per_second': 19.619, 'train_steps_per_second': 2.476, 'train_loss': 1.2969607848387499, 'epoch': 10.0}


TrainOutput(global_step=390, training_loss=1.2969607848387499, metrics={'train_runtime': 157.4999, 'train_samples_per_second': 19.619, 'train_steps_per_second': 2.476, 'train_loss': 1.2969607848387499, 'epoch': 10.0})

In [36]:

def post_process(output_sequences):
    predictions = []
    generated_sequences = []

    max_repeat = 2

    # decode prediction
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
        generated_sequences.append(text.strip())
                    
    for i, g in enumerate(generated_sequences):
        res = str(g).replace('\n\n\n', '\n').replace('\n\n', '\n')
        lines = res.split('\n')
        # print(lines)
        i = max_repeat
        while i != len(lines):
          remove_count = 0
          for index in range(0, max_repeat):
            # print(i - index - 1, i - index)
            if lines[i - index - 1] == lines[i - index]:
              remove_count += 1
          if remove_count == max_repeat:
            lines.pop(i)
            i -= 1
          else:
            i += 1
        predictions.append('\n'.join(lines))

    return predictions

In [37]:
prompt = "Hey there "
tokenized_prompt = tokenizer(prompt, return_tensors='pt', add_special_tokens=False).input_ids.to(trainer.args.device)
output = trainer.model.generate(
                        input_ids=tokenized_prompt,
                        max_length=100,
                        min_length=60,
                        temperature=float(1),
                        top_p=float(0.95),
                        top_k=int(50),
                        do_sample=True,
                        repetition_penalty=1,
                        num_return_sequences=40
                        )

print(post_process(output))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["Hey there!!!\nWhat a waste\nI was\nI was there\nI remember you dancing before your eyes went wild\nOn backroads at night\nWhen I passed the pictures around of you\nAnd your little eyelids flutter cause it's glow\nIn the winter\nBut I watched it rain\nOh, it's just so quiet in the world\nI was watching you\nIt was just you, I was watching you\nI watched you\nI", "Hey there \nWell I guess I wished on a plane I thought was a star\nHey, yeah\nYeah\nI guess I liked that\nYou liked that\nWell I guess you liked that\nThat little black dress\nI wore before you went and let me down\nThere you stand now, ten feet tall\nEulogize me\nCause I'm the tallest building we had ever been\nI was in the tallest building I was in the whole scene", "Hey there \nI hope you see me\nLookin' like a face in the crowd\nI hope you see me\nOoh look what you made me do\nLook like a face in the crowd\nI'd give up forever for\nI'd give up forever to touch you\nBut I'm so sorry\nI don't see\n you here\nI'd meet you he

In [11]:

trainer.push_to_hub("trained")

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
Upload file pytorch_model.bin: 320MB [02:34, 4.84MB/s]                            To https://huggingface.co/BhavyaMuni/model-generator
   94d8184..a638e07  main -> main

Upload file pytorch_model.bin: 100%|██████████| 318M/318M [02:35<00:00, 2.15MB/s]
Upload file training_args.bin: 100%|██████████| 4.30k/4.30k [02:35<00:00, 28.3B/s]
To https://huggingface.co/BhavyaMuni/model-generator
   a638e07..e23e1db  main -> main



'https://huggingface.co/BhavyaMuni/model-generator/commit/a638e076a80f154ef8053473b9d58c29a4acd27d'